# Assignment 5 Machine Translation
### Jing Qian (jq2282)

## Step 1. Install packages and load libraries


In [1]:
!pip install -q tensorflow-gpu==2.0.0-alpha0
!pip install sacrebleu # https://github.com/mjpost/sacreBLEU

In [0]:
import numpy as np
import re
import sacrebleu
import tensorflow as tf
import time
import unicodedata

## Step 2. Load and preprocess data

### 2.1. Load and select data

In [0]:
ntrain = 10000
ntest = 1000
nall = ntrain + ntest
nepoch = 100

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
def load(fname):
  # Load the file using std open
  f = open(fname, 'r')
  text = []
  for line in f.readlines():
    text.append(line.replace('\n','').split('\t'))
    
  f.close()
  return text

data = load('/content/gdrive/My Drive/spa-eng/spa.txt')

In [6]:
print(data[10:15])
print(np.shape(data))

[['Fire!', '¡Disparad!'], ['Help!', '¡Ayuda!'], ['Help!', '¡Socorro! ¡Auxilio!'], ['Help!', '¡Auxilio!'], ['Jump!', '¡Salta!']]
(118964, 2)


In [7]:
#np.random.seed(10)
shuffled_data = np.random.permutation(data)
selected_id = np.random.randint(len(data), size = nall)
train_data = shuffled_data[selected_id[:ntrain], :]
print(np.shape(train_data), np.shape(shuffled_data))

(10000, 2) (118964, 2)


### 2.2. Preprocessing

In [0]:
def preprocess(s):
  # for details, see https://www.tensorflow.org/alpha/tutorials/sequences/nmt_with_attention
  s = ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
  s = re.sub(r"([?.!,¿])", r" \1 ", s)
  s = re.sub(r'[" "]+', " ", s)
  s = re.sub(r"[^a-zA-Z?.!,¿]+", " ", s)
  s = s.strip()
  s = '<start> ' + s + ' <end>'
  return s

In [9]:
train_data = [(preprocess(eng), preprocess(spa)) for (eng, spa) in train_data]
print(train_data[0])
train_eng, train_spa = list(zip(*train_data))

('<start> These are my friends . <end>', '<start> Estos son mis amigos . <end>')


In [10]:
eng_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
eng_tokenizer.fit_on_texts(train_eng)
train_eng = eng_tokenizer.texts_to_sequences(train_eng)
train_eng = tf.keras.preprocessing.sequence.pad_sequences(train_eng, padding='post')
print(train_eng[0])

[  1 233  30  22 226   3   2   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]


In [11]:
spa_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
spa_tokenizer.fit_on_texts(train_spa)
train_spa = spa_tokenizer.texts_to_sequences(train_spa)
train_spa = tf.keras.preprocessing.sequence.pad_sequences(train_spa, padding='post')
print(train_spa[0])

[  1 294  62 124 215   3   2   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0]


In [0]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
spa_vocab_size = len(spa_tokenizer.word_index) + 1

In [13]:
# Demonstrate the correspondence between word and code, not used in model
def decode(encoded, tokenizer):
  for number in encoded:
    if number !=0:
      print ("%d -> %s" % (number, tokenizer.index_word[number]))
      
decode(train_eng[0], eng_tokenizer)

1 -> <start>
233 -> these
30 -> are
22 -> my
226 -> friends
3 -> .
2 -> <end>


## Step 3. Train Model 1: translate from English to Spanish


### 3.1. Prepare data for model1

In [14]:
# Create labels for the decoder by shifting the target sequence
# one to the right.
target_labels = np.zeros(train_spa.shape)
target_labels[:,0:train_spa.shape[1] -1] = train_spa[:,1:]

print("Target sequence", train_spa[0])
print("Target label", target_labels[0])

Target sequence [  1 294  62 124 215   3   2   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0]
Target label [294.  62. 124. 215.   3.   2.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]


In [0]:
batch_size = 5
dataset = tf.data.Dataset.from_tensor_slices((train_eng, train_spa, target_labels)).batch(batch_size)

In [0]:
# Test code!
# example_batch = next(iter(dataset))
# source, target, taget_labels = example_batch
# print("Shapes:", source.shape, target.shape, taget_labels.shape)

### 3.2. General components for models

In [0]:
embedding_size = 32
rnn_size = 64

In [0]:
# Differ from example, add source_vocab_size to initialization
class Encoder(tf.keras.Model):
  def __init__(self, source_vocab_size):
    super(Encoder, self).__init__()
    
    self.embedding = tf.keras.layers.Embedding(source_vocab_size,
                                               embedding_size)
    self.gru = tf.keras.layers.GRU(rnn_size, 
                                   return_sequences=True, 
                                   return_state=True)
    
  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state=hidden)        
    return output, state
  
  def init_state(self, batch_size):
    return tf.zeros((batch_size, rnn_size))

In [0]:
# Test code!
# Create a batch of one sentence
# ex_sentence = tf.expand_dims(train_eng[0], axis=0)
# ex_translation = tf.expand_dims(train_spa[0], axis=0)
# ex_labels = tf.expand_dims(target_labels[0], axis=0)
# print(ex_sentence.shape)


In [0]:
# Differ from example, add target_vocab_size to initialization
class Decoder(tf.keras.Model):
  def __init__(self, target_vocab_size):
    super(Decoder, self).__init__()
    self.embedding = tf.keras.layers.Embedding(target_vocab_size, 
                                               embedding_size)
    self.gru = tf.keras.layers.GRU(rnn_size, 
                                   return_sequences=True, 
                                   return_state=True)

    self.dense = tf.keras.layers.Dense(target_vocab_size)


  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state=hidden)
    logits = self.dense(output)
    return logits, state

In [0]:
crossentropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

def calc_loss(targets, logits):
  mask = tf.math.logical_not(tf.math.equal(targets, 0))
  mask = tf.cast(mask, dtype=tf.int64)
  return crossentropy(targets, logits, sample_weight=mask)

#print("Loss", calc_loss(ex_labels, decoder_output))

In [0]:
def translate(idx=None, flag=1):
  
    if idx == None: 
      idx = np.random.choice(len(train_data))
    
    if flag == 1:
      source_data = train_eng
      target_tokenizer = spa_tokenizer
      encoder = encoder1
      decoder = decoder1
      tmp0, tmp1 = train_data[idx][0], train_data[idx][1]
    elif flag == 2:
      source_data = train_spa
      target_tokenizer = eng_tokenizer
      encoder = encoder2
      decoder = decoder2
      tmp0, tmp1 = train_data[idx][1], train_data[idx][0] 
    elif flag == 3:
      source_data = test_eng
      target_tokenizer = spa_tokenizer
      encoder = encoder1
      decoder = decoder1
      tmp0, tmp1 = test_data[idx][0], test_data[idx][1]
    elif flag == 4:
      source_data = input_spa
      target_tokenizer = eng_tokenizer
      encoder = encoder2
      decoder = decoder2
      tmp0, tmp1 = middle_spa[idx], test_data[idx][0]       
      
    input_sent = source_data[idx]
    input_sent = tf.expand_dims(input_sent, axis=0)
    
    hidden_state = encoder.init_state(batch_size=1)
    output, hidden_state = encoder(input_sent, hidden_state)
    
    decoder_input = tf.expand_dims([target_tokenizer.word_index['<start>']], 0)
    out_words = []
    
    decoder_state = hidden_state

    while True:
      
        decoder_output, decoder_state = decoder(decoder_input, decoder_state)
        decoder_input = tf.argmax(decoder_output, -1)
        word_idx = decoder_input.numpy()[0][0]
        # if we've predicted 0 (which is reserved, usually this will only happen
        # before the decoder is trained, just stop translating and return
        # what we have)
        if word_idx == 0: 
          out_words.append('<end>')
        else:
          out_words.append(target_tokenizer.index_word[word_idx])

        if out_words[-1] == '<end>' or len(out_words) >= 20:
          break
          
    translation = ' '.join(out_words)    
    return tmp0, tmp1, translation

In [0]:
# Test code
# input_sent, target_sent, translation = translate()
# print("Input: %s\nTarget: %s\nTranslation: %s\n" % (input_sent, target_sent, translation))

In [0]:
optimizer = tf.keras.optimizers.Adam()

### 3.3. Train Model 1: from English to Spanish

In [0]:
@tf.function # remove this annotation when debugging
def train_step(source_seq, target_seq, target_labels, initial_state):
  
  with tf.GradientTape() as tape:
    encoder_output, encoder_state = encoder1(source_seq, initial_state)
    logits, decoder_state = decoder1(target_seq, encoder_state)
    loss = calc_loss(target_labels, logits)

  variables = encoder1.trainable_variables + decoder1.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))

  return loss

In [25]:
EPOCHS = nepoch

encoder1 = Encoder(eng_vocab_size)
decoder1 = Decoder(spa_vocab_size)

for epoch in range(EPOCHS):
    start = time.time()
  
    en_initial_states = encoder1.init_state(batch_size)
    
    for batch, (source_seq, target_seq, target_labels) in enumerate(dataset):
      loss = train_step(source_seq, target_seq, target_labels, en_initial_states)
      elapsed = time.time() - start
    
    if epoch % 10 == 0:
      print("Epoch #%d, Loss %.4f, Time %.2f sec" % (epoch, loss, elapsed))
      input_sent, target_sent, translation = translate()
      print("Input: %s\nTarget: %s\nTranslation: %s\n" % (input_sent, target_sent, translation))

Epoch #0, Loss 0.9821, Time 25.76 sec
Input: <start> Can anyone answer my question ? <end>
Target: <start> ¿ Alguien puede contestar mi pregunta ? <end>
Translation: ¿ no me gusta que el que el que el que el que el que el que el que el

Epoch #10, Loss 0.5638, Time 18.85 sec
Input: <start> Tom wants a new coat . <end>
Target: <start> Tom quiere un abrigo nuevo . <end>
Translation: tom se quedo en boston . <end>

Epoch #20, Loss 0.4031, Time 19.01 sec
Input: <start> Where are all the others ? <end>
Target: <start> ¿ Donde estan todos los demas ? <end>
Translation: ¿ donde esta tom ? <end>

Epoch #30, Loss 0.3387, Time 18.75 sec
Input: <start> Why are you so worried ? <end>
Target: <start> ¿ Por que estas tan preocupada ? <end>
Translation: ¿ por que no quieres comer ? <end>

Epoch #40, Loss 0.2955, Time 20.04 sec
Input: <start> Tom s strategy paid off . <end>
Target: <start> La estrategia de Tom surtio efecto . <end>
Translation: la gente de tom le explico su un coche nueva a su coche .

In [26]:
# Test code for BLEU score
references, hypotheses = [], []

for i in range(20):
  input_sent, target_sent, translation = translate()
  references.append(target_sent)
  hypotheses.append("<start> " + translation)
  
results = sacrebleu.raw_corpus_bleu(hypotheses, [references])
print(results)

BLEU(score=10.95967477564419, counts=[82, 31, 8, 6], totals=[202, 182, 162, 142], precisions=[40.59405940594059, 17.032967032967033, 4.938271604938271, 4.225352112676056], bp=1.0, sys_len=202, ref_len=183)


## Step 4. Train Model 2: translate from Spanish to English

### 4.1. Prepare data for model2


In [0]:
# Create labels for the decoder by shifting the target sequence
# one to the right.
target_labels = np.zeros(train_eng.shape)
target_labels[:,0:train_eng.shape[1] -1] = train_eng[:,1:]
dataset = tf.data.Dataset.from_tensor_slices((train_spa, train_eng, target_labels)).batch(batch_size)

### 4.2 Train Model2: from Spanish to English

In [0]:
@tf.function # remove this annotation when debugging
def train_step2(source_seq, target_seq, target_labels, initial_state):
  
  with tf.GradientTape() as tape:
    encoder_output, encoder_state = encoder2(source_seq, initial_state)
    logits, decoder_state = decoder2(target_seq, encoder_state)
    loss = calc_loss(target_labels, logits)

  variables = encoder2.trainable_variables + decoder2.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))

  return loss

In [29]:
encoder2 = Encoder(spa_vocab_size)
decoder2 = Decoder(eng_vocab_size)

for epoch in range(EPOCHS):
    start = time.time()
  
    en_initial_states = encoder2.init_state(batch_size)
    
    for batch, (source_seq, target_seq, target_labels) in enumerate(dataset):
      loss = train_step2(source_seq, target_seq, target_labels, en_initial_states)
      elapsed = time.time() - start
    
    if epoch % 10 == 0:
      print("Epoch #%d, Loss %.4f, Time %.2f sec" % (epoch, loss, elapsed))
      input_sent, target_sent, translation = translate(flag=2)
      print("Input: %s\nTarget: %s\nTranslation: %s\n" % (input_sent, target_sent, translation))

Epoch #0, Loss 0.9542, Time 21.57 sec
Input: <start> Casi nadie sabe que somos hermanas . <end>
Target: <start> Almost no one knows we are sisters . <end>
Translation: i m not a good . <end>

Epoch #10, Loss 0.6111, Time 18.34 sec
Input: <start> Kioto fue la antigua capital de Japon . <end>
Target: <start> Kyoto was the former capital of Japan . <end>
Translation: i m not going to see you . <end>

Epoch #20, Loss 0.5528, Time 18.28 sec
Input: <start> La chica que toca el piano es mi hermana . <end>
Target: <start> The girl who s playing the piano is my sister . <end>
Translation: i m not going to help . <end>

Epoch #30, Loss 0.5253, Time 19.30 sec
Input: <start> El puso su casa en venta . <end>
Target: <start> He put up his house for sale . <end>
Translation: i m not going to help . <end>

Epoch #40, Loss 0.5136, Time 17.98 sec
Input: <start> Tom podia oir al gato rasgunando la puerta . <end>
Target: <start> Tom could hear the cat scratching the door . <end>
Translation: i m not going

In [30]:
# Test code for BLEU score
references, hypotheses = [], []

for i in range(20):
  input_sent, target_sent, translation = translate(flag=2)
  references.append(target_sent)
  hypotheses.append("<start> " + translation)
  
results = sacrebleu.raw_corpus_bleu(hypotheses, [references])
print(results)

BLEU(score=0.0, counts=[78, 23, 1, 0], totals=[212, 192, 172, 152], precisions=[36.79245283018868, 11.979166666666666, 0.5813953488372093, 0.0], bp=1.0, sys_len=212, ref_len=196)


## Step 5. Back-translate
Use your two models to translate a sentence from English to Spanish, and then back to English. Compare the original sentence, and the back-translated sentence. Repeat this using an evaluation corpus of 1,000 sentences, and report the BLEU score.

### 5.1. Preprocess test data

In [0]:
test_data = shuffled_data[selected_id[ntrain:], :]
test_data = [(preprocess(eng), preprocess(spa)) for (eng, spa) in test_data]
#print(train_data[0])
test_eng, test_spa = list(zip(*test_data))
test_eng = eng_tokenizer.texts_to_sequences(test_eng)
test_eng = tf.keras.preprocessing.sequence.pad_sequences(test_eng, padding='post')

### 5.2. Using Model 1 to translate test data from English to Spanish.

In [0]:
middle_spa = []
origin_eng = []
for i in range(ntest):
  input_sent, target_sent, translation = translate(idx=i, flag=3)
  #print(input_sent)
  origin_eng.append(input_sent)
  middle_spa.append(translation[:-5])

### 5.3. Using Model 2 to translate the output from Model 1 back to English

In [0]:
input_spa = spa_tokenizer.texts_to_sequences(middle_spa)
input_spa = tf.keras.preprocessing.sequence.pad_sequences(input_spa, padding='post')
#print(input_spa)

In [0]:
back_eng = []
for i in range(ntest):
  input_sent, target_sent, translation = translate(idx=i, flag=4)
  #print(translation)
  back_eng.append("<start> " + translation)

### 5.4. Calculate BLEU

In [35]:
results = sacrebleu.raw_corpus_bleu(back_eng, [origin_eng])
print(results)  

BLEU(score=2.432903795755739, counts=[3465, 992, 34, 14], totals=[9727, 8727, 7727, 6727], precisions=[35.622494088619305, 11.367021886100607, 0.4400155299598809, 0.2081165452653486], bp=0.9859128520180013, sys_len=9727, ref_len=9865)
